In [115]:
import content_extractor

import configparser

from importlib import reload

# set the max columns to none
import pandas as pd
pd.set_option('display.max_columns', None)
# set max_colwidth to 3000
pd.set_option('display.max_colwidth', 100)

In [129]:
reload(content_extractor)

config = configparser.ConfigParser()

config_file_path = 'config/test.ini'
config.read(config_file_path)

# Get configuration options
input_filename = config.get('General', 'input_filename')
output_filename = config.get('General', 'output_filename')
output_filename = None if output_filename == "None" else output_filename
mode = config.get('General', 'mode')
num_processes = config.getint('General', 'num_processes')
url_col_name = config.get('General', 'url_col_name')
pub_date_col_name = config.get('General', 'pub_date_col_name')

if mode in {'openai', 'all'}:
    openai_model = config.get('General', 'openai_model')
    openai_temp = config.getfloat('General', 'openai_temp')
    openai_max_tokens = config.getint('General', 'openai_max_tokens') 
    
    # Initialize ContentExtractor
    extractor = content_extractor.ContentExtractor(openai_model, openai_temp, openai_max_tokens)

elif mode == 'extractor': extractor = content_extractor.ContentExtractor()
    

In [130]:
# Read data
data_df = extractor.read_data(input_filename, url_col_name=url_col_name, pub_date_col_name=pub_date_col_name)
data_df

,PublishedDate,Language,Alert_definition,PT,Title,Content,Id,URL,Summary,New_Content,Is_Article
0,2023-10-05T07:50:39Z,en,NaN,NaN,NaN,NaN,NaN,https://www.cbc.ca/news/canada/newfoundland-labrador/post-tropical-storm-nicole-port-aux-basques...,Port aux Basques experienced minor flooding Saturday as the remnants of post-tropical storm Nico...,Port aux Basques experienced minor flooding Saturday as the remnants of post-tropical storm Nico...,1
1,2023-10-05T07:50:39Z,en,NaN,NaN,NaN,NaN,NaN,https://www.saltwire.com/atlantic-canada/news/post-tropical-storm-earl-leaves-damaged-st-johns-r...,"ST. JOHN'S, N.L. St. Johns resident Steve Shortall was up at 6 oclock Monday morning, Sept. 12, ...","ST. JOHN'S, N.L. St. Johns resident Steve Shortall was up at 6 oclock Monday morning, Sept. 12, ...",1
2,2023-10-05T07:50:39Z,en,NaN,NaN,NaN,NaN,NaN,https://www.winnipegfreepress.com/our-communities/west/overland-flooding-soaks-municipalities-57...,"Headingley mayor John Mauseth said Dodds Road, on which he himself lives, saw the worst of the f...","Hey there, time traveller! This article was published 03/05/2022 (568 days ago), so information ...",1
3,2023-10-05T07:50:39Z,en,NaN,NaN,NaN,NaN,NaN,https://www.cbc.ca/news/canada/manitoba/manitoba-flooding-evacuations-emergency-1.6437263,"As Manitoba experiences its third straight weekend of challenging weather, some households are e...","As Manitoba experiences its third straight weekend of challenging weather, some households are e...",1
4,2023-10-05T07:50:39Z,en,NaN,NaN,NaN,NaN,NaN,https://winnipeg.ctvnews.ca/overland-flooding-sparks-state-of-emergency-in-r-m-of-dufferin-1.588...,Several Manitoba municipalities have declared a local State of Emergency due to ongoing flooding...,Several Manitoba municipalities have declared a local State of Emergency due to ongoing flooding...,1


In [125]:
url_content_df = extractor.extract_content(data_df, num_processes=num_processes, out_fn=output_filename)

INFO:root:Saving results ...
INFO:root:Saved.


In [114]:
url_content_df

,PublishedDate,Language,Alert_definition,PT,Title,Content,Id,URL,Summary,New_Content,Is_Article
0,2023-10-05T07:50:39Z,en,NaN,NaN,NaN,NaN,NaN,https://www.cbc.ca/news/canada/newfoundland-labrador/post-tropical-storm-nicole-port-aux-basques...,Port aux Basques experienced minor flooding Saturday as the remnants of post-tropical storm Nico...,Port aux Basques experienced minor flooding Saturday as the remnants of post-tropical storm Nico...,1
1,2023-10-05T07:50:39Z,en,NaN,NaN,NaN,NaN,NaN,https://www.saltwire.com/atlantic-canada/news/post-tropical-storm-earl-leaves-damaged-st-johns-r...,"ST. JOHN'S, N.L. St. Johns resident Steve Shortall was up at 6 oclock Monday morning, Sept. 12, ...","ST. JOHN'S, N.L. St. Johns resident Steve Shortall was up at 6 oclock Monday morning, Sept. 12, ...",1
2,2023-10-05T07:50:39Z,en,NaN,NaN,NaN,NaN,NaN,https://www.winnipegfreepress.com/our-communities/west/overland-flooding-soaks-municipalities-57...,"Headingley mayor John Mauseth said Dodds Road, on which he himself lives, saw the worst of the f...","Hey there, time traveller! This article was published 03/05/2022 (568 days ago), so information ...",1
3,2023-10-05T07:50:39Z,en,NaN,NaN,NaN,NaN,NaN,https://www.cbc.ca/news/canada/manitoba/manitoba-flooding-evacuations-emergency-1.6437263,"As Manitoba experiences its third straight weekend of challenging weather, some households are e...","As Manitoba experiences its third straight weekend of challenging weather, some households are e...",1
4,2023-10-05T07:50:39Z,en,NaN,NaN,NaN,NaN,NaN,https://winnipeg.ctvnews.ca/overland-flooding-sparks-state-of-emergency-in-r-m-of-dufferin-1.588...,Several Manitoba municipalities have declared a local State of Emergency due to ongoing flooding...,Several Manitoba municipalities have declared a local State of Emergency due to ongoing flooding...,1


In [122]:
# Run example om a sigle URL with OpenAI

tmp_df = url_content_df.iloc[0]
openai_content_0 = extractor.extract_single_event_chatopenai(tmp_df['New_Content'], tmp_df['URL'], tmp_df['PublishedDate'], openai_model=openai_model, openai_temp=openai_temp, openai_max_tokens=openai_max_tokens)
# content_df = extractor.transform_openai_response_to_df(openai_content_0)
# content_df

INFO:content_extractor:Extracting information from https://www.cbc.ca/news/canada/newfoundland-labrador/post-tropical-storm-nicole-port-aux-basques-1.6650252
INFO:content_extractor:Transforming information from https://www.cbc.ca/news/canada/newfoundland-labrador/post-tropical-storm-nicole-port-aux-basques-1.6650252
ERROR:content_extractor:An error occurred during transformation: expected string or bytes-like object, got 'DataFrame'


""


In [123]:
openai_content_0

,is_happened,flood_cause_en,date,location,death,evacuation,country,link,published_date
0,Yes,Heavy rain from the remnants of post-tropical storm Nicole,Unknown,"Port aux Basques, Newfoundland",Unknown,Unknown,Canada,https://www.cbc.ca/news/canada/newfoundland-labrador/post-tropical-storm-nicole-port-aux-basques...,2023-10-05T07:50:39Z


In [59]:
# Run example om a sigle URL with OpenAI

tmp_df = url_content_df.iloc[1]
openai_content = extractor.extract_single_event_chatopenai(tmp_df['New_Content'], tmp_df['URL'], openai_model=openai_model, openai_temp=openai_temp, openai_max_tokens=openai_max_tokens)

INFO:content_extractor:Extracting information from https://www.saltwire.com/atlantic-canada/news/post-tropical-storm-earl-leaves-damaged-st-johns-roads-and-flooded-properties-in-its-wake-100772111/#:~:text=Post%2Dtropical%20storm%20Earl%20dumped,it%20was%20impassable%20Monday%20morning.
INFO:content_extractor:Transforming information from https://www.saltwire.com/atlantic-canada/news/post-tropical-storm-earl-leaves-damaged-st-johns-roads-and-flooded-properties-in-its-wake-100772111/#:~:text=Post%2Dtropical%20storm%20Earl%20dumped,it%20was%20impassable%20Monday%20morning.


In [61]:
content_df = extractor.transform_openai_response_to_df(openai_content)

content_df

,is_happened,flood_cause_en,date,location,death,evacuation,country
0,Yes,Heavy rainfall from post-tropical storm Earl,2022-09,"St. John's, Mount Pearl, Newfoundland and Labrador",Unknown,Unknown,Canada


In [53]:
# Run example om a sigle URL with OpenAI

tmp_df = url_content_df.iloc[3]
openai_content_3 = extractor.extract_single_event_chatopenai(tmp_df['New_Content'], tmp_df['URL'], openai_model=openai_model, openai_temp=openai_temp, openai_max_tokens=openai_max_tokens)

INFO:content_extractor:Extracting information from https://www.cbc.ca/news/canada/manitoba/manitoba-flooding-evacuations-emergency-1.6437263
INFO:content_extractor:Transforming information from https://www.cbc.ca/news/canada/manitoba/manitoba-flooding-evacuations-emergency-1.6437263


In [54]:
content_df = extractor.transform_openai_response_to_df(openai_content_3)

content_df

,is_happened,flood_cause_en,date,location,death,evacuation,country
0,Yes,Heavy rain and waterlogged ground,Unknown,"[Ritchot, Manitoba, Morden, Manitoba, Fisher River Cree Nation, Manitoba, Winnipeg, Manitoba]",Unknown,Unknown,Canada


In [57]:
tmp_df = url_content_df.iloc[2]
openai_content_2 = extractor.extract_single_event_chatopenai(tmp_df['New_Content'], tmp_df['URL'], openai_model=openai_model, openai_temp=openai_temp, openai_max_tokens=openai_max_tokens)

INFO:content_extractor:Extracting information from https://www.winnipegfreepress.com/our-communities/west/overland-flooding-soaks-municipalities-576463972.html
INFO:content_extractor:Transforming information from https://www.winnipegfreepress.com/our-communities/west/overland-flooding-soaks-municipalities-576463972.html


In [58]:
content_df = extractor.transform_openai_response_to_df(openai_content_2)

content_df

,is_happened,flood_cause_en,date,location,death,evacuation,country
0,Yes,Heavy snowfall followed by rainstorm,2022-05,"Headingley (Dodds Road), RM of Headingley, RM of Rosser, RM of MacDonald, RM of Cartier",Unknown,Unknown,Canada
